In [634]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [635]:

# 데이터 증강 및 정규화를 위한 변환 설정
# train데이터는 데이터 증식이 필요
# 데이터 증식 : 과적합을 피하기 위함
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 데이터 증강: 랜덤하게 잘라내기
    transforms.RandomHorizontalFlip(),      # 데이터 증강: 랜덤하게 좌우 반전
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  # 정규화 rgb 평균갑 / rgb 표준편차값
])


# 테스트 데이터는 데이터 증식을 하면 안된다.
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# CIFAR10 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)


# train/valid data 분리하기
train_ratio = 0.8
train_size = int(len(train_dataset) * train_ratio)
val_size = len(train_dataset) - train_size
train_dataset, valid_dataset = random_split(train_dataset, [train_size, val_size])


# 데이터 로더 생성
batch_size = 128  # 원하는 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)    #num_worckers -> cpu 개수 / 이미지가 커지면 4~8개까지 줘야함 / 코랩에서는 2개이상 줄수없다.
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [636]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 입력 이미지(3,32,32)
# 출력 이미지(0~9)
classes = 10

class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()
        # 합성곱 레이어 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 입력 채널: 3, 출력 채널: 32
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화

        # 합성곱 레이어 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 입력 채널: 32, 출력 채널: 64
        self.bn2 = nn.BatchNorm2d(64)

        # 합성곱 레이어 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 입력 채널: 64, 출력 채널: 128
        self.bn3 = nn.BatchNorm2d(128)

        # 완전 연결 레이어
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # 128 채널의 4x4 특성 맵을 512 유닛으로 변환
        self.fc2 = nn.Linear(512, classes)  # 512 유닛을 10개의 클래스로 변환

    def forward(self, x):
        # 합성곱 레이어 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 합성곱 레이어 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 합성곱 레이어 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # 맥스 풀링

        # 완전 연결 레이어
        x = x.view(-1, 128 * 4 * 4)  # 텐서를 1차원으로 펼치기
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [637]:
# 모델 정의 (3채널 입력, CIFAR-10에 맞게 수정)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 합성곱 층
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 3채널 입력
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # 완전 연결 층
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # CIFAR-10은 32x32 이미지, MaxPool 후 16x16
        self.fc2 = nn.Linear(128, 10)  # CIFAR-10은 10개의 클래스
        # 활성화 함수와 드롭아웃
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.relu(self.conv1(x))   # [batch_size, 32, 32, 32]
        x = self.relu(self.conv2(x))   # [batch_size, 64, 32, 32]
        x = self.pool(x)               # [batch_size, 64, 16, 16]
        x = self.dropout(x)
        x = x.view(-1, 64 * 16 * 16)   # 평탄화
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [638]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [639]:
print(device)

cuda


In [640]:
lr = 0.001
epochs = 50

# EfficientNet 모델 불러오기 (사전 학습된 가중치 사용)
model = EfficientNet.from_pretrained('efficientnet-b0')

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 마지막 레이어 변경 (CIFAR-10은 10개의 클래스)
num_classes = 10
model._fc = nn.Linear(model._fc.in_features, num_classes)  # 마지막 레이어를 CIFAR-10에 맞게 변경
model = model.to(device)
# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 알고리즘 정의
optimizer = optim.Adam(model.parameters(), lr=lr)

# 디버깅: 모델 파라미터가 GPU로 이동했는지 확인
for name, param in model.named_parameters():
    print(f"Layer: {name} is on {param.device}")

Loaded pretrained weights for efficientnet-b0
Layer: _conv_stem.weight is on cuda:0
Layer: _bn0.weight is on cuda:0
Layer: _bn0.bias is on cuda:0
Layer: _blocks.0._depthwise_conv.weight is on cuda:0
Layer: _blocks.0._bn1.weight is on cuda:0
Layer: _blocks.0._bn1.bias is on cuda:0
Layer: _blocks.0._se_reduce.weight is on cuda:0
Layer: _blocks.0._se_reduce.bias is on cuda:0
Layer: _blocks.0._se_expand.weight is on cuda:0
Layer: _blocks.0._se_expand.bias is on cuda:0
Layer: _blocks.0._project_conv.weight is on cuda:0
Layer: _blocks.0._bn2.weight is on cuda:0
Layer: _blocks.0._bn2.bias is on cuda:0
Layer: _blocks.1._expand_conv.weight is on cuda:0
Layer: _blocks.1._bn0.weight is on cuda:0
Layer: _blocks.1._bn0.bias is on cuda:0
Layer: _blocks.1._depthwise_conv.weight is on cuda:0
Layer: _blocks.1._bn1.weight is on cuda:0
Layer: _blocks.1._bn1.bias is on cuda:0
Layer: _blocks.1._se_reduce.weight is on cuda:0
Layer: _blocks.1._se_reduce.bias is on cuda:0
Layer: _blocks.1._se_expand.weight is

In [641]:
# lr = 0.001
# epochs = 50
# # 모델정의
# model = Net().to(device)
# # 손실합수
# criterion = nn.CrossEntropyLoss()
# # 최적화 알고리즘
# optimizer = optim.Adam(model.parameters(), lr=lr)

In [642]:
train_losses = []
valid_losses = []
train_accs = []
valid_accs = []

In [643]:
# print("train")
print("\033[34mtrain")


train


In [644]:
# def train(model, train_loader, criterion, optimizer, epoch):
#     model.train()
#     train_loss = 0
#     train_correct = 0
#     train_total = 0
#     train_accuracy = 0

#     startStr = f'Epoch : {epoch+1:2d} /{epochs} \033[34m' + ' Train' + '\033[0m'
#     print(startStr)
#     with tqdm(total=len(train_loader), desc=startStr) as pbar:
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)

#             train_total += labels.size(0)
#             train_correct += (predicted == labels).sum().item()
#             pbar.update(1)

#         train_loss = train_loss/ len(train_loader)
#         train_accuracy = train_correct / train_total
#         train_losses.append(train_loss)
#         train_accs.append(train_accuracy)
#         endStr = f'Loss : {train_loss / (pbar.n + 1):.4f} Acc : {train_correct/train_total:.4f}'
#         print(endStr)

In [645]:
best_val_acc = 0.0  # 최고 검증 정확도를 추적
best_epoch = 0  # 최고 검증 정확도를 기록한 에포크를 추적
best_model_path = 'best_model.pth'  # 모델을 저장할 파일 경로

In [646]:
def train(model, device, train_loader, optimizer, epoch, epochs):
    # 훈련 모드 설정
    model.train()

    train_loss=0
    train_correct=0
    train_total=0

    startStr =f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Train'+ '\033[0m'
    with tqdm(total=len(train_loader), desc=startStr) as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            pbar.update(1)
        train_loss = train_loss / train_total
        train_accuracy = (train_correct / train_total)*100
        train_losses.append(train_loss)
        train_accs.append(train_accuracy)
        endStr = f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}%'
        # tqdm동작이 다 끝나면 그때 아래의 코드가 동작
        print(endStr)
        #pbar.set_postfix(endStr)
        
    return train_loss, train_accuracy 

In [647]:
def valid_or_test(mode, model, device, dataloader, epoch, epochs):
    global best_val_acc, best_epoch  # 전역 변수로 사용
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        if mode == 'valid':
            startStr = f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Valid ' + '\033[0m'
        elif mode == 'test':
            startStr = f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Test ' + '\033[0m'

        with tqdm(total=len(dataloader), desc=startStr) as pbar:
            for data, target in dataloader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                loss = criterion(outputs, target)

                loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                pbar.update(1)

            loss = loss / total
            accuracy = 100 * correct / total
            if mode == 'valid':
                endStr = f'Valid Loss: {loss:.4f}, Valid Acc: {accuracy:.3f}%'
            elif mode == 'test':
                endStr = f'Test Loss: {loss:.4f}, Test Acc: {accuracy:.3f}%'
            pbar.set_postfix_str(endStr)

    if mode == 'valid':
        valid_losses.append(loss)
        valid_accs.append(accuracy)
        print('-'*110)
        
    return loss, accuracy

In [648]:
for name, param in model.named_parameters():
    print(f"Layer: {name} is on {param.device}")

Layer: _conv_stem.weight is on cuda:0
Layer: _bn0.weight is on cuda:0
Layer: _bn0.bias is on cuda:0
Layer: _blocks.0._depthwise_conv.weight is on cuda:0
Layer: _blocks.0._bn1.weight is on cuda:0
Layer: _blocks.0._bn1.bias is on cuda:0
Layer: _blocks.0._se_reduce.weight is on cuda:0
Layer: _blocks.0._se_reduce.bias is on cuda:0
Layer: _blocks.0._se_expand.weight is on cuda:0
Layer: _blocks.0._se_expand.bias is on cuda:0
Layer: _blocks.0._project_conv.weight is on cuda:0
Layer: _blocks.0._bn2.weight is on cuda:0
Layer: _blocks.0._bn2.bias is on cuda:0
Layer: _blocks.1._expand_conv.weight is on cuda:0
Layer: _blocks.1._bn0.weight is on cuda:0
Layer: _blocks.1._bn0.bias is on cuda:0
Layer: _blocks.1._depthwise_conv.weight is on cuda:0
Layer: _blocks.1._bn1.weight is on cuda:0
Layer: _blocks.1._bn1.bias is on cuda:0
Layer: _blocks.1._se_reduce.weight is on cuda:0
Layer: _blocks.1._se_reduce.bias is on cuda:0
Layer: _blocks.1._se_expand.weight is on cuda:0
Layer: _blocks.1._se_expand.bias is

In [649]:
best_val_acc = 0.0
best_epoch = 0
best_model_path = 'best_model.pth'  # 저장할 모델 파일 이름
# 훈련
for epoch in range(epochs):
 # 매 epoch마다 가중치를(파라미터) 학습
    train_loss, train_accuracy = train(model, train_loader, optimizer,epoch,epochs)
    # 매 epoch마다 loss값을 출력
    print(f"epoch : {epoch +1}/{epochs}, train loss : {train_loss/len(train_loader)}")

    # 검증 단계
    valid_loss, val_acc = valid_or_test(valid_loader, model, valid_loader, epoch,epochs)
    print(f"valid loss : {valid_loss/len(valid_loader)}, accuracy : {val_acc}")
    
    # 성능이 가장 좋은 에포크에서 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1  # 1-based 인덱스
        torch.save(model.state_dict(), best_model_path)  # 모델 가중치 저장
        print(f"Best model saved at epoch {best_epoch} with accuracy {best_val_acc}")

# 모델 학습이 끝난 후, 가장 성능이 좋았던 에포크를 출력
print(f"Training complete. Best model was at epoch {best_epoch} with accuracy {best_val_acc}.")

TypeError: train() missing 1 required positional argument: 'epochs'

In [ ]:
# # 테스트
# valid_or_test('test', model, device, test_loader, epoch)



In [ ]:
# 최고 검증 정확도를 기록한 에포크 및 모델 저장 경로 출력
print(f"Training complete. Best model was saved at epoch {best_epoch} with accuracy {best_val_acc:.3f}%.")

# 테스트 (저장된 최고의 모델 사용)
model.load_state_dict(torch.load(best_model_path))
valid_or_test('test', model, device, test_loader, epoch,epochs)

In [ ]:
# import matplotlib.pyplot as plt

# # 손실 그래프 그리기
# def plot_loss(train_losses, valid_losses):
#     plt.figure(figsize=(10, 5))
#     plt.plot(train_losses, label='Train Loss')
#     plt.plot(valid_losses, label='Valid Loss')
#     plt.title('Loss Graph')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# # 정확도 그래프 그리기
# def plot_accuracy(train_accs, valid_accs):
#     plt.figure(figsize=(10, 5))
#     plt.plot(train_accs, label='Train Accuracy')
#     plt.plot(valid_accs, label='Valid Accuracy')
#     plt.title('Accuracy Graph')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy (%)')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# # 훈련 및 검증 후 손실 및 정확도 그래프 그리기
# plot_loss(train_losses, valid_losses)
# plot_accuracy(train_accs, valid_accs)

In [ ]:
import matplotlib.pyplot as plt

# 텐서를 CPU로 이동하고 넘파이로 변환하는 함수
def tensor_to_numpy(tensor_list):
    return [t.cpu().numpy() if isinstance(t, torch.Tensor) else t for t in tensor_list]

# 손실 그래프 그리기
def plot_loss(train_losses, valid_losses):
    # 텐서를 CPU로 옮기고 넘파이로 변환
    train_losses = tensor_to_numpy(train_losses)
    valid_losses = tensor_to_numpy(valid_losses)
    
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Valid Loss')
    plt.title('Loss Graph')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# 정확도 그래프 그리기
def plot_accuracy(train_accs, valid_accs):
    # 텐서를 CPU로 옮기고 넘파이로 변환
    train_accs = tensor_to_numpy(train_accs)
    valid_accs = tensor_to_numpy(valid_accs)

    plt.figure(figsize=(10, 5))
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(valid_accs, label='Valid Accuracy')
    plt.title('Accuracy Graph')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    plt.show()

# 훈련 및 검증 후 손실 및 정확도 그래프 그리기
plot_loss(train_losses, valid_losses)
plot_accuracy(train_accs, valid_accs)